# From Python to Production
## Notebook 8 — Object-Oriented Programming (OOP)

By **Prerna Joshi** | #25DaysOfDataTech 

"OOP models the real world — structure your code like systems, not scripts."

---

### What you'll learn
- Classes, instances, attributes, and methods
- `__init__`, `__repr__`, `__str__`, rich comparisons (`__eq__`, `__lt__`)
- Class vs instance variables; `@classmethod`, `@staticmethod`
- Properties for validation and lazy computation
- Composition vs inheritance; mixins; ABCs
- Data classes: fast, clean, and typed records
- Copying, equality, hashing; immutability patterns (`frozen=True`)
- Slots for memory efficiency; pitfalls to avoid


> **Why this matters for data work**  
> OOP shines when you need **stateful components** (parsers, validators, model wrappers, services). Done right, it improves structure and testability.


## 1. Class Basics — Instances & Methods


In [1]:
class User:
    def __init__(self, user_id: int, name: str, role: str = "member"):
        self.user_id = user_id
        self.name = name
        self.role = role

    def greet(self) -> str:
        return f"Hello, {self.name} ({self.role})"

u = User(101, "Prerna", role="Data Engineer")
u.greet(), vars(u)


('Hello, Prerna (Data Engineer)',
 {'user_id': 101, 'name': 'Prerna', 'role': 'Data Engineer'})

## 2. `__repr__` vs `__str__`

- `__repr__`: unambiguous, for developers (ideally valid Python to rebuild the object)
- `__str__`: pretty, for users


In [2]:
class Point:
    def __init__(self, x: float, y: float):
        self.x, self.y = x, y
    def __repr__(self):
        return f"Point(x={self.x!r}, y={self.y!r})"
    def __str__(self):
        return f"({self.x}, {self.y})"

p = Point(3, 4)
repr(p), str(p)


('Point(x=3, y=4)', '(3, 4)')

## 3. Equality, Ordering, Hashing

Implement `__eq__` to compare **state**, and `__hash__` if instances must be dict/set keys (must be consistent with equality). For ordering, define `__lt__` or use `functools.total_ordering`.


In [3]:
from functools import total_ordering

@total_ordering
class Score:
    def __init__(self, name: str, value: int):
        self.name, self.value = name, value
    def __eq__(self, other):
        if not isinstance(other, Score): return NotImplemented
        return (self.value, self.name) == (other.value, other.name)
    def __lt__(self, other):
        if not isinstance(other, Score): return NotImplemented
        return (self.value, self.name) < (other.value, other.name)
    def __hash__(self):
        return hash((self.value, self.name))

s1 = Score("alice", 91)
s2 = Score("bob", 91)
sorted([s2, s1]), {s1, s2}


([<__main__.Score at 0x211d5a616d0>, <__main__.Score at 0x211d5b93110>],
 {<__main__.Score at 0x211d5a616d0>, <__main__.Score at 0x211d5b93110>})

## 4. Class vs Instance Variables; `@classmethod` & `@staticmethod`


In [4]:
class Config:
    # Class variable (shared); don't store mutable defaults here unless intentional
    DEFAULT_REGION = "us-east-1"
    instances = 0

    def __init__(self, region=None):
        self.region = region or self.DEFAULT_REGION
        Config.instances += 1

    @classmethod
    def with_region(cls, region):
        return cls(region=region)

    @staticmethod
    def is_valid_region(region):
        return isinstance(region, str) and len(region) >= 3

c1 = Config(); c2 = Config.with_region("eu-west-1")
(Config.instances, c1.region, c2.region, Config.is_valid_region("ap-south-1"))


(2, 'us-east-1', 'eu-west-1', True)

## 5. Properties — Validation, Caching, and Computed Attributes


In [5]:
class Product:
    def __init__(self, price: float, tax_rate: float = 0.1):
        self._price = None
        self.tax_rate = tax_rate
        self.price = price  # triggers validation

    @property
    def price(self) -> float:
        return self._price

    @price.setter
    def price(self, value: float):
        if value < 0:
            raise ValueError("price must be non-negative")
        self._price = float(value)

    @property
    def total(self) -> float:
        return round(self.price * (1 + self.tax_rate), 2)

p = Product(100)
p.total, (setattr(p, "price", 149.5) or p.total)


(110.0, 164.45)

## 6. Inheritance vs Composition

- **Prefer composition** to extend behavior by combining smaller objects.  
- Use **inheritance** for clear "is‑a" relationships and override points.


In [6]:
class CSVReader:
    def read(self, path):
        with open(path, encoding="utf-8") as f:
            return f.read().splitlines()

class DataService:
    def __init__(self, reader: CSVReader):
        self.reader = reader   # composition
    def head(self, path, n=3):
        return self.reader.read(path)[:n]

# inheritance example
class SpecialReader(CSVReader):
    def read(self, path):
        data = super().read(path)
        return [line.strip() for line in data if line and not line.startswith("#")]


## 7. Mixins — Share Small Behaviors

Mixins provide focused methods that rely on a host class's API. They shouldn't be instantiated alone.


In [7]:
class ReprMixin:
    def __repr__(self):
        fields = ", ".join(f"{k}={v!r}" for k, v in sorted(vars(self).items()))
        return f"{self.__class__.__name__}({fields})"

class Person(ReprMixin):
    def __init__(self, name, email):
        self.name, self.email = name, email

Person("Prerna", "pj@example.com")


Person(email='pj@example.com', name='Prerna')

## 8. Abstract Base Classes (ABCs) — Contracts


In [8]:
from abc import ABC, abstractmethod

class Model(ABC):
    @abstractmethod
    def predict(self, X):
        ...

class MeanModel(Model):
    def __init__(self, mean):
        self.mean = mean
    def predict(self, X):
        return [self.mean for _ in X]

MeanModel(3.14).predict([1,2,3])


[3.14, 3.14, 3.14]

## 9. Data Classes — Clean, Typed Records


In [9]:
from dataclasses import dataclass, field
from typing import List

@dataclass(order=True, frozen=False)
class Student:
    score: int
    name: str
    tags: List[str] = field(default_factory=list, compare=False)

alice = Student(91, "alice", ["math"])
bob = Student(88, "bob")
(alice, bob, alice > bob, alice.tags.append("ml") or alice)


(Student(score=91, name='alice', tags=['math', 'ml']),
 Student(score=88, name='bob', tags=[]),
 True,
 Student(score=91, name='alice', tags=['math', 'ml']))

## 10. Immutability & Hashing

Use `frozen=True` to make instances immutable (and hashable if fields are hashable). For custom classes, ensure `__hash__` aligns with `__eq__`.


In [10]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Key:
    user_id: int
    env: str = "prod"

k1 = Key(1, "prod"); k2 = Key(1, "prod")
{k1: "ok"}, k1 == k2, hash(k1) == hash(k2)


({Key(user_id=1, env='prod'): 'ok'}, True, True)

## 11. `__slots__` — Reduce Memory & Prevent Typos

`__slots__` restricts attributes to a fixed set and avoids a per-instance `__dict__` (saves memory for many objects).


In [11]:
class Row:
    __slots__ = ("id", "value")
    def __init__(self, id, value):
        self.id = id
        self.value = value

r = Row(1, 3.14)
(hasattr(r, "__dict__"), r.id, r.value)


(False, 1, 3.14)

## 12. Copying & Pitfalls

- Beware **mutable default class attributes** (shared across instances).  
- For deep structures, use `copy.deepcopy` to avoid shared inner objects.


In [12]:
import copy

class Bucket:
    items = []  # class attribute (shared!) — usually a pitfall
    def __init__(self, items=None):
        self.items = list(items or [])  # safe: new list each time

b1 = Bucket([1]); b2 = Bucket()
Bucket.items.append(99)   # affects class-level list only
(b1.items, b2.items, Bucket.items)


([1], [], [99])

## 13. Type Hints & Protocols (Duck Typing)

Use `typing.Protocol` to define structural contracts (method signatures) without inheritance.


In [13]:
from typing import Protocol, Iterable

class Sized(Protocol):
    def __len__(self) -> int: ...

def has_data(x: Sized) -> bool:
    return len(x) > 0

has_data([1,2,3]), has_data("abc"), has_data(range(0))


(True, True, False)

## 14. Mini Cheatsheet

- Prefer composition; inherit only for true is‑a relations
- Implement `__repr__` for debugging and logs
- Use properties for validation and computed fields
- Reach for `dataclasses` for record‑like objects
- Consider `__slots__` for many small objects
- Keep equality/hash consistent; avoid mutable keys


## 15. Practice (Try first, then reveal solutions)

1. **BankAccount**: Class with `deposit`, `withdraw` (validate: non‑negative, sufficient funds), `balance` property (read‑only).  
2. **Vector2D**: Implement `__repr__`, `__add__`, `__sub__`, `__eq__`, and magnitude method.  
3. **from_csv (classmethod)**: Add to a `User`‑like class to construct from `"id,name,role"` string.  
4. **SlugifyMixin**: Mixin that adds `.slug()` using lower + dash joins; integrate into a `Post` class.  
5. **CacheModel**: ABC with `.predict(X)`; add a concrete class that caches results by hashable input.  
6. **Student (dataclass)**: `order=True`, default `tags=[]` via `default_factory`, and a computed property `is_honors` (`score >= 90`).  
7. **@property validation**: `Temperature.celsius` with validation; computed `fahrenheit`.  
8. **Registry**: Class with a class‑level registry dict; `register(cls, obj)` classmethod and `get(name)` lookup.  
9. **Resource**: Implement `__enter__/__exit__` context manager that opens/closes a file safely.  
10. **ImmutableKey**: Frozen dataclass used as dict key; show equality/hash behavior.  
11. **SlotsRow**: Class with `__slots__` = `("id","name","score")`; show that adding a new attribute raises `AttributeError`.  
12. **Protocol demo**: Define a `SupportsScore` protocol with `.score` attribute and a function that works on any such object.


## 16. Practice Solutions  
*(Click to reveal after solving.)*

<details>
<summary><strong>Solution 1️⃣ — BankAccount</strong></summary>

```python
class BankAccount:
    def __init__(self, opening=0.0):
        self._balance = float(opening)
    @property
    def balance(self):
        return self._balance
    def deposit(self, amt):
        if amt < 0: raise ValueError("negative deposit")
        self._balance += amt
    def withdraw(self, amt):
        if amt < 0: raise ValueError("negative withdraw")
        if amt > self._balance: raise ValueError("insufficient funds")
        self._balance -= amt
```
</details>

<details>
<summary><strong>Solution 2️⃣ — Vector2D</strong></summary>

```python
class Vector2D:
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __repr__(self):
        return f"Vector2D({self.x!r}, {self.y!r})"
    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)
    def __sub__(self, other):
        return Vector2D(self.x - other.x, self.y - other.y)
    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)
    def mag(self):
        return (self.x**2 + self.y**2) ** 0.5
```
</details>

<details>
<summary><strong>Solution 3️⃣ — from_csv</strong></summary>

```python
class UserX:
    def __init__(self, user_id, name, role="member"):
        self.user_id, self.name, self.role = int(user_id), name, role
    @classmethod
    def from_csv(cls, line: str):
        user_id, name, role = [x.strip() for x in line.split(",")]
        return cls(user_id, name, role)
```
</details>

<details>
<summary><strong>Solution 4️⃣ — SlugifyMixin</strong></summary>

```python
class SlugifyMixin:
    def slug(self):
        return "-".join(self.title.lower().split())

class Post(SlugifyMixin):
    def __init__(self, title, body):
        self.title, self.body = title, body
```
</details>

<details>
<summary><strong>Solution 5️⃣ — CacheModel</strong></summary>

```python
from abc import ABC, abstractmethod

class CacheModel(ABC):
    @abstractmethod
    def predict(self, X): ...

class EchoCache(CacheModel):
    def __init__(self):
        self._cache = {}
    def predict(self, X):
        if X in self._cache:
            return self._cache[X]
        y = X  # pretend compute
        self._cache[X] = y
        return y
```
</details>

<details>
<summary><strong>Solution 6️⃣ — Student dataclass</strong></summary>

```python
from dataclasses import dataclass, field

@dataclass(order=True)
class Student:
    score: int
    name: str
    tags: list[str] = field(default_factory=list, compare=False)
    @property
    def is_honors(self):
        return self.score >= 90
```
</details>

<details>
<summary><strong>Solution 7️⃣ — Temperature</strong></summary>

```python
class Temperature:
    def __init__(self, celsius=0.0):
        self.celsius = celsius
    @property
    def celsius(self):
        return self._c
    @celsius.setter
    def celsius(self, v):
        v = float(v)
        if v < -273.15: raise ValueError("below absolute zero")
        self._c = v
    @property
    def fahrenheit(self):
        return self._c * 9/5 + 32
```
</details>

<details>
<summary><strong>Solution 8️⃣ — Registry</strong></summary>

```python
class Registry:
    _reg = {}
    @classmethod
    def register(cls, name, obj):
        cls._reg[name] = obj
    @classmethod
    def get(cls, name, default=None):
        return cls._reg.get(name, default)
```
</details>

<details>
<summary><strong>Solution 9️⃣ — Resource</strong></summary>

```python
class Resource:
    def __init__(self, path, mode="r", encoding="utf-8"):
        self.path, self.mode, self.encoding = path, mode, encoding
        self._f = None
    def __enter__(self):
        self._f = open(self.path, self.mode, encoding=self.encoding)
        return self._f
    def __exit__(self, exc_type, exc, tb):
        if self._f:
            self._f.close()
        return False  # don't suppress exceptions
```
</details>

<details>
<summary><strong>Solution 🔟 — ImmutableKey</strong></summary>

```python
from dataclasses import dataclass

@dataclass(frozen=True)
class ImmutableKey:
    id: int
    name: str
# works as dict key
```
</details>

<details>
<summary><strong>Solution 1️⃣1️⃣ — SlotsRow</strong></summary>

```python
class SlotsRow:
    __slots__ = ("id","name","score")
    def __init__(self, id, name, score):
        self.id, self.name, self.score = id, name, score
# Trying: r.extra = 5 -> AttributeError
```
</details>

<details>
<summary><strong>Solution 1️⃣2️⃣ — Protocol demo</strong></summary>

```python
from typing import Protocol

class SupportsScore(Protocol):
    score: int

def is_top(x: SupportsScore, threshold=90) -> bool:
    return x.score >= threshold
```
</details>
